<h1>TP 4 AI52 : Algorithme généritque quantique</h1>


<h2>Importations</h2>

In [2]:
import numpy as np
from copy import copy

<h2>Définition du problème</h2>

In [3]:
class BackpackProblem():
    def __init__(self,capacity,number_of_objects,min_weight,max_weight,min_value,max_value) -> None:
        self.capacity = capacity
        self.number_of_objects = number_of_objects
        self.min_weight = min_weight
        self.max_weight = max_weight
        self.min_value = min_value
        self.max_value = max_value

        self.item_values = [0]*number_of_objects
        self.item_weights = [0]*number_of_objects

        self.generate()

    def generate(self):
        while sum(self.item_weights) < 3*self.capacity:
            for i in range(self.number_of_objects):
                self.item_values[i] = np.random.randint(self.min_value,self.max_value)
                self.item_weights[i] = np.random.randint(self.min_weight,self.max_weight)

    def printProblem(self):
        print("Liste des objets disponibles : ")
        for i in range(self.number_of_objects):
            print("Objet n°",i,", poids =",self.item_weights[i],", valeur =",self.item_values[i])

    def printSolution(self,solution):
        if len(solution) != self.number_of_objects:
            print("ERROR IN SOLUTION")
            exit(0)
        print("Objets du sac à dos : ")
        for i in range(self.number_of_objects):
            if solution[i] == 1:
                print("Objet n°",i)

        total_value = 0
        total_weight = 0

        for i in range(self.number_of_objects) :
            total_value += solution[i]*self.item_values[i]
            total_weight += solution[i]*self.item_weights[i]

        print("Valeur totale : ",total_value)
        print("Poids total : ",total_weight)
        print("Fitness : ",self.fitness(solution))
        

    
    def fitness(self,solution):
        if len(solution) != self.number_of_objects:
            print("ERROR IN SOLUTION")
            exit(0)
        total_value = 0
        total_weight = 0

        for i in range(self.number_of_objects) :
            total_value += solution[i]*self.item_values[i]
            total_weight += solution[i]*self.item_weights[i]

        if total_weight <= self.capacity:
            return total_value
        
        return self.capacity - total_weight
    



<h2>Algorithme génétique</h2>

In [43]:

def geneticAlgorithm(problem: BackpackProblem,population_size,generations):
    population = []
    for _ in range(population_size):
        chromosome = []
        for _ in range(problem.number_of_objects):
            chromosome.append(np.random.randint(0,2))
        population.append(chromosome)
    
    for _ in  range(generations):

        for i in range(population_size):
            population.append(mutate(population[i]))
            population.append(uniformRandomCrossover(population[i],population[(i+1)%population_size]))
        population.sort(key=lambda x: problem.fitness(x),reverse=True)

        population = population[:population_size]
    
    return population[0]

def mutate(chromosome):
    child = copy(chromosome)
    mutation_count = np.random.randint(1,4)
    for _ in range(mutation_count):
        mutation_target = np.random.randint(0,len(child))
        child[mutation_target] = 1-child[mutation_target]
    return child

def uniformRandomCrossover(chr1,chr2):
    return [np.random.choice(i) for i in zip(chr1,chr2)]

<h2>Algorithme quantique</h2>

In [33]:
class Qubit():
    def __init__(self,angle = np.pi/4) -> None:
        self.alpha = np.cos(angle)
        self.beta = np.sin(angle)

    def collapse(self):
        if np.random.random() < pow(self.alpha,2):
            return 0
        return 1
    
    def UGate(self,angle):
        new = copy(self)
        newAlpha = self.alpha*np.cos(angle) + self.beta*np.sin(angle)
        newBeta = -self.alpha*np.sin(angle) + self.beta*np.cos(angle)
        new.alpha, new.beta = newAlpha,newBeta
        return new
    
def rotationAngle(x: bool, b: bool, better_fitness: bool, signe_a: int, signe_b: int):
    if not x and (not b or (b and not better_fitness)):
        delta_y = 0
        s = 0
    elif b and (x != better_fitness):
        delta_y = 0.05*np.pi
        if signe_a*signe_b != 0:
            if better_fitness:
                s = signe_a*signe_b
            else:
                s = -signe_a*signe_b
        else:
            #TODO : +-1 stuff
            pass
    elif x and better_fitness:
        delta_y = 0.025*np.pi
        if signe_a*signe_b != 0:
            if b:
                s = -signe_a*signe_b
            else:
                s = signe_a*signe_b
        else:
            #TODO : +-1 stuff
            pass
    else:
        delta_y = 0.01*np.pi
        if signe_a*signe_b != 0:
            s = signe_a*signe_b
        else:
            #TODO : +-1 stuff
            pass
    
    return delta_y*s
    

     

def QuanticAlgorithm(problem : BackpackProblem, population_size = 10, generations = 10):
    quantum_population = []
    for _ in range(population_size):
        quantum_chromosome = []
        for _ in range(problem.number_of_objects):
            quantum_chromosome.append(Qubit(np.pi/4))
        quantum_population.append(quantum_chromosome)
    
    collapsed_population = []
    for quantum_chromosome in quantum_population:
        collapsed_chromosome = []
        for qubit in quantum_chromosome:
            collapsed_chromosome.append(qubit.collapse())
        collapsed_population.append(collapsed_chromosome)
    
    collapsed_population.sort(key=lambda x: problem.fitness(x),reverse=True)

    best = collapsed_population[0]

    for t in generations :
        for j in range(population_size):
            for i in range(problem.number_of_objects):
                angle = rotationAngle(
                    collapsed_population[j][i],
                    best[i],
                    problem.fitness(collapsed_population[j]) >= problem.fitness(best),
                    np.sign(quantum_population[i][j].alpha),
                    np.sign(quantum_population[i][j].beta)
                )
                quantum_population[j][i] = quantum_population[i][j].UGate(angle)
        
        collapsed_population = []
        for quantum_chromosome in quantum_population:
            collapsed_chromosome = []
            for qubit in quantum_chromosome:
                collapsed_chromosome.append(qubit.collapse())
            collapsed_population.append(collapsed_chromosome)
        
        collapsed_population.sort(key=lambda x: problem.fitness(x),reverse=True)

        best = collapsed_population[0]
    
    return best
    

    


    


<h2>Comparaison sur différentes instances :</h2>

In [47]:
NOMBRE_INSTANCES = 5
CAPACITY = 30
OBJECT_COUNT = 10
MIN_VALUE = 1
MAX_VALUE = 100
MIN_WEIGHT = 1
MAX_WEIGHT = 25

GENETIC_POP_SIZE = 20
GENETIC_GENERATIONS = 50

for i in range(NOMBRE_INSTANCES):
    problem = BackpackProblem(CAPACITY,OBJECT_COUNT,MIN_WEIGHT,MAX_WEIGHT,MIN_VALUE,MAX_VALUE)
    solution_genetique = geneticAlgorithm(problem,GENETIC_POP_SIZE,GENETIC_GENERATIONS)
    print("\nInstance",i)
    print("Problème :")
    problem.printProblem()
    print("Meilleure solution genetique : ")
    problem.printSolution(solution_genetique)

    


Instance 0
Problème :
Liste des objets disponibles : 
Objet n° 0 , poids = 12 , valeur = 74
Objet n° 1 , poids = 2 , valeur = 66
Objet n° 2 , poids = 23 , valeur = 70
Objet n° 3 , poids = 15 , valeur = 18
Objet n° 4 , poids = 16 , valeur = 80
Objet n° 5 , poids = 5 , valeur = 74
Objet n° 6 , poids = 20 , valeur = 30
Objet n° 7 , poids = 11 , valeur = 31
Objet n° 8 , poids = 21 , valeur = 18
Objet n° 9 , poids = 12 , valeur = 33
Meilleure solution genetique : 
Objets du sac à dos : 
Objet n° 0
Objet n° 1
Objet n° 5
Objet n° 7
Valeur totale :  245
Poids total :  30
Fitness :  245

Instance 1
Problème :
Liste des objets disponibles : 
Objet n° 0 , poids = 22 , valeur = 69
Objet n° 1 , poids = 8 , valeur = 93
Objet n° 2 , poids = 2 , valeur = 36
Objet n° 3 , poids = 15 , valeur = 19
Objet n° 4 , poids = 2 , valeur = 77
Objet n° 5 , poids = 21 , valeur = 15
Objet n° 6 , poids = 11 , valeur = 60
Objet n° 7 , poids = 9 , valeur = 29
Objet n° 8 , poids = 13 , valeur = 38
Objet n° 9 , poids = 